In [2]:
import xarray as xr

ds = xr.open_dataset("GIEMS-MC_compressed.nc")
print(ds)


<xarray.Dataset> Size: 3GB
Dimensions:                      (time: 348, latitude: 720, longitude: 1440,
                                  month: 12)
Coordinates:
  * time                         (time) datetime64[ns] 3kB 1992-01-01 ... 202...
  * latitude                     (latitude) float32 3kB -89.88 -89.62 ... 89.88
  * longitude                    (longitude) float32 6kB -179.9 -179.6 ... 179.9
  * month                        (month) int8 12B 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    inund_sat_wetland_frac       (time, latitude, longitude) float32 1GB ...
    inund_sat_peat_wetland_frac  (time, latitude, longitude) float32 1GB ...
    fresh_lake_frac              (latitude, longitude) float32 4MB ...
    saline_lake_frac             (latitude, longitude) float32 4MB ...
    reservoir_frac               (latitude, longitude) float32 4MB ...
    river_frac                   (latitude, longitude) float32 4MB ...
    estu_river_frac              (latitude, longitude) float32 4M

In [13]:
import xarray as xr
import numpy as np
import os

# === CONFIGURATION ===
input_file = "GIEMS-MC_compressed.nc"
output_file = "WETLAND_Canada_Annual_2010_2024.nc"
var_name = "inund_sat_wetland_frac"
new_var_name = "wetland_fraction"

# ✅ Define bounding box for Canada
canada_bounds = dict(
    lat_min=40, lat_max=85,  # Latitude goes from North to South in GIEMS-MC
    lon_min=-141, lon_max=-52
)

# === STEP 1: Load and Subset ===
print("🔹 Loading dataset...")
ds = xr.open_dataset(input_file)

# ✅ Correct slicing: descending latitude in GIEMS-MC
# ds = ds.sel(
#     latitude=slice(canada_bounds["lat_max"], canada_bounds["lat_min"]),
#     longitude=slice(canada_bounds["lon_min"], canada_bounds["lon_max"])
# )

# ✅ Correct slicing: use ascending latitude in GIEMS-MC
ds = ds.sel(
    latitude=slice(canada_bounds["lat_min"], canada_bounds["lat_max"]),  # 40 → 85
    longitude=slice(canada_bounds["lon_min"], canada_bounds["lon_max"])  # -141 → -52
)


print("DEBUG: Latitude size after slicing:", ds.latitude.size)
print("DEBUG: Longitude size after slicing:", ds.longitude.size)


# Keep only the variable of interest
wet = ds[var_name]

# === STEP 2: Monthly to Annual Mean (2010–2021) ===
print("🔹 Calculating annual means (2010–2021)...")
wet = wet.sel(time=slice("2010-01-01", "2021-12-31"))
annual = wet.groupby("time.year").mean("time")

# === STEP 3: AR(2) Forecast Function ===
def forecast_ar2(series):
    y = series.copy()
    for _ in range(3):  # Predict 2022, 2023, 2024
        if len(y) < 2 or np.isnan(y[-1]) or np.isnan(y[-2]):
            y = np.append(y, np.nan)
        else:
            # AR(2): y[t+1] = a*y[t] + b*y[t-1]
            a, b = 0.7, 0.2
            y_next = a * y[-1] + b * y[-2]
            y = np.append(y, y_next)
    return y[-3:]

# === STEP 4: Forecast for All Grid Points ===
print("🔹 Forecasting 2022–2024...")
pred_years = [2022, 2023, 2024]
pred_stack = []

for i, year in enumerate(pred_years):
    forecasted = xr.apply_ufunc(
        lambda x: forecast_ar2(x)[i],
        annual,
        input_core_dims=[["year"]],
        vectorize=True,
        dask="parallelized",
        output_dtypes=[float]
    )
    forecasted = forecasted.expand_dims(year=[year])
    pred_stack.append(forecasted)

# Combine historical + forecast
forecast_ds = xr.concat(pred_stack, dim="year")
full_annual = xr.concat([annual, forecast_ds], dim="year")
full_annual.name = new_var_name

# === STEP 5: Save Output ===
print("🔹 Merging and saving...")
output_ds = full_annual.to_dataset()
output_ds.to_netcdf(output_file)

print(f"\n✅ Done: Saved to {output_file}")


🔹 Loading dataset...
DEBUG: Latitude size after slicing: 180
DEBUG: Longitude size after slicing: 356
🔹 Calculating annual means (2010–2021)...
🔹 Forecasting 2022–2024...
🔹 Merging and saving...

✅ Done: Saved to WETLAND_Canada_Annual_2010_2024.nc
